<a href="https://colab.research.google.com/github/JosueAfouda/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SEGMENTING AND CLUSTERING OF NEIGHBORHOODS IN TORONTO**

*Author : Josué Afouda*

# Table of Contents
**[Introduction](#M0)** 

**[Get the data by scraping Wikipedia page](#M1)**   

**[Latitude and Longitude coordinates of each neighborhood](#M2)**

**[Exploring of the neighborhoods in Toronto](#M3)**

**[Clustering of the neighborhoods in Toronto](#M4)**

**[Neighborhoods for each cluster](#M5)**

# <font color=#3876C2> Introduction</font> <a name="M0"></a>

In this notebook, **i will explore, segment, and cluster the neighborhoods in the city of Toronto**. However, the neighborhood data is not readily available on the internet. For the Toronto neighborhood data, a [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) exists that has all the information i need to explore and cluster the neighborhoods in Toronto. So, i will **scrape** the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

In [1]:
! pip install wikipedia

In [0]:
#Libraries
import pandas as pd
import numpy as np
import wikipedia as wp

# <font color=#3876C2> Get the data by scraping Wikipedia page</font> <a name="M1"></a>

In [3]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('postal_codes_toronto.csv',header=0,index=False)
print (df)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


In [0]:
#Rename 'Postcode'
df.rename(columns={"Postcode":"PostalCode"}, inplace=True)

In [5]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
PostalCode       210 non-null object
Borough          210 non-null object
Neighbourhood    210 non-null object
dtypes: object(3)
memory usage: 5.0+ KB


**The dataframe is clean**.

In [7]:
print('My dataframe contains', df.shape[0], 'rows and', df.shape[1], 'columns')

My dataframe contains 210 rows and 3 columns


# <font color=#3876C2> Latitude and Longitude coordinates of each neighborhood</font> <a name="M2"></a>

In [0]:
coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [9]:
coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [0]:
#Rename 'Postcode'
coord.rename(columns={"Postal Code":"PostalCode"}, inplace=True)

In [11]:
#Merge df and coord dataframes on 'Postal Code'
data = pd.merge(df, coord, on="PostalCode")
data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
206,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999


In [12]:
data.isna().sum()

PostalCode       0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

# <font color=#3876C2> Exploring of the neighborhoods in Toronto</font> <a name="M3"></a>

In [0]:
#Libraries
import json 
import geopy
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [14]:
#Latitude and Longitude of Toronto city
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of the Toronto city are : {}, {}.'.format(latitude, longitude))

The geographical coordinates of the Toronto city are : 43.653963, -79.387207.


In [15]:
# create map of Toronto with neighborhoods superimposed on top using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

#Print map
map_toronto

In [0]:
#My Foursquare Credentials
CLIENT_ID = 'RSIO4BSJMK1BSCICX3LTPX0KP4RMR0CTAWYVU4DQI4N0R3A5' # your Foursquare ID
CLIENT_SECRET = 'LP2HUAG21MJT1TQWO2U3PVRQYGDHNFMUC5GQ33RLXL4WLC3O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Now, let's get the top 100 venues that are in each neighborhood of Toronto in a radius of 500 meters.

In [0]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, let's write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [18]:
toronto_venues = getNearbyVenues(names=data['Neighbourhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toro

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Let's check how many venues were returned for each neighborhood.

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Albion Gardens,8,8,8,8,8,8
Alderwood,10,10,10,10,10,10
...,...,...,...,...,...,...
Woodbine Gardens,11,11,11,11,11,11
Woodbine Heights,10,10,10,10,10,10
York Mills West,3,3,3,3,3,3


How many unique categories can be curated from all the returned venues ?

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


Let's analyze Each Neighborhood.

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,...,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
#The new dataframe size
toronto_onehot.shape

(4314, 270)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,...,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.02,0.000000,0.0,0.000000,0.0,0.0,0.02,0.000000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.020000,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.01,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.250000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.100000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,...,0.0,0.1,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Woodbine Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.090909,0.0,0.000000,0.0,0.0,0.00,0.090909,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.090909,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
200,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.100,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,...,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,

In [24]:
#Confirm the new size
toronto_grouped.shape

(204, 270)

Let's print each neighborhood along with the top 5 most common venues.

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2              Café  0.04
3   Thai Restaurant  0.04
4  Sushi Restaurant  0.03


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2         Chinese Restaurant  0.25
3  Latin American Restaurant  0.25
4                Yoga Studio  0.00


----Agincourt North----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Albion Gardens----
                  venue  freq
0         Grocery Store  0.25
1            Beer Store  0.12
2        Sandwich Place  0.12
3   Fried Chicken Joint  0.12
4  Fast Food Restaurant  0.12


----Alderwood----
          venue  freq
0   Pizza Place   0.2
1           Pub   0.1
2      Pharmacy   0.1
3           Gym   0.1
4  Skating Rink   0.1


--

Let's put that into a pandas dataframe.

First, let's write a function to sort the venues in descending order.


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Sushi Restaurant,Asian Restaurant,Breakfast Spot,Burger Joint,Seafood Restaurant
1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Chinese Restaurant,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
2,Agincourt North,Playground,Park,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
3,Albion Gardens,Grocery Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,German Restaurant,Dance Studio,Dumpling Restaurant
4,Alderwood,Pizza Place,Pharmacy,Coffee Shop,Sandwich Place,Pub,Athletics & Sports,Pool,Skating Rink,Gym,Construction & Landscaping


# <font color=#3876C2> Clustering of the neighborhoods in Toronto</font> <a name="M4"></a>

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [29]:
# add clustering labels
labels = kmeans.labels_
neighborhoods_venues_sorted["Cluster Labels"] = labels

toronto_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Food & Drink Shop,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,2.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,1.0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,Coffee Shop,Pub,Park,Café,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Farmers Market,1.0
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,Clothing Store,Accessories Store,Event Space,Vietnamese Restaurant,Furniture / Home Store,Boutique,Miscellaneous Shop,Coffee Shop,Arts & Crafts Store,Women's Store,1.0
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,Clothing Store,Accessories Store,Event Space,Vietnamese Restaurant,Furniture / Home Store,Boutique,Miscellaneous Shop,Coffee Shop,Arts & Crafts Store,Women's Store,1.0


# <font color=#3876C2> Neighborhoods for each cluster</font> <a name="M5"></a>

### Cluster 0

In [30]:
#Neighborhoods of Cluster 0
toronto_neighborhood_cluster0 = toronto_merged[toronto_merged["Cluster Labels"] == 0].Neighbourhood
toronto_neighborhood_cluster0

105                       Emery
106                   Humberlea
197                  Humber Bay
198            King's Mill Park
199    Kingsway Park South East
200                   Mimico NE
201              Old Mill South
202          The Queensway East
203       Royal York South East
204                    Sunnylea
Name: Neighbourhood, dtype: object

### Cluster 1

In [31]:
#Neighborhoods of Cluster 1
toronto_neighborhood_cluster1 = toronto_merged[toronto_merged["Cluster Labels"] == 1].Neighbourhood
toronto_neighborhood_cluster1

1              Victoria Village
2                  Harbourfront
3              Lawrence Heights
4                Lawrence Manor
5                  Queen's Park
                 ...           
205    Kingsway Park South West
206                   Mimico NW
207          The Queensway West
208       Royal York South West
209              South of Bloor
Name: Neighbourhood, Length: 174, dtype: object

### Cluster 2

In [32]:
#Neighborhoods of Cluster 2
toronto_neighborhood_cluster2 = toronto_merged[toronto_merged["Cluster Labels"] == 2].Neighbourhood
toronto_neighborhood_cluster2

0                Parkwoods
37     Caledonia-Fairbanks
53     Scarborough Village
149             Moore Park
150        Summerhill East
154        Agincourt North
155        L'Amoreaux East
156               Milliken
157           Steeles East
182               Rosedale
Name: Neighbourhood, dtype: object

### Cluster 3

In [33]:
#Neighborhoods of Cluster 3
toronto_neighborhood_cluster3 = toronto_merged[toronto_merged["Cluster Labels"] == 3].Neighbourhood
toronto_neighborhood_cluster3

20     Highland Creek
21         Rouge Hill
22         Port Union
101           Del Ray
102        Keelesdale
103      Mount Dennis
104       Silverthorn
Name: Neighbourhood, dtype: object

### Cluster 4

In [34]:
#Neighborhoods of Cluster 4
toronto_neighborhood_cluster4 = toronto_merged[toronto_merged["Cluster Labels"] == 4].Neighbourhood
toronto_neighborhood_cluster4

15          Cloverdale
16           Islington
17        Martin Grove
18    Princess Gardens
19     West Deane Park
Name: Neighbourhood, dtype: object